In [1]:
from sage.all import *
from scipy.special import logit

In [2]:
n_arms = 2
ts = [var(f't_{i}') for i in range(n_arms)]
mu_0 = -1.34
mu_0 = var('mu_0')
logit_p1 = logit(0.3)
logit_p1 = var('L')

Q = [[var(f'Q_{i}{j}') for j in range(n_arms)] for i in range(n_arms)]

y = [var(f'y_{i}') for i in range(4)]
n = [var(f'n_{i}') for i in range(4)]

In [3]:
sig2 = var('S')
cov = [[100 for i in range(4)] for j in range(4)]
for i in range(4):
    cov[i][i] += sig2

In [4]:
Qrough = matrix(cov).inverse()
Q = [[Qrough[i][j].full_simplify() for i in range(4)] for j in range(4)]
Q

[[(S + 300)/(S^2 + 400*S),
  -100/(S^2 + 400*S),
  -100/(S^2 + 400*S),
  -100/(S^2 + 400*S)],
 [-100/(S^2 + 400*S),
  (S + 300)/(S^2 + 400*S),
  -100/(S^2 + 400*S),
  -100/(S^2 + 400*S)],
 [-100/(S^2 + 400*S),
  -100/(S^2 + 400*S),
  (S + 300)/(S^2 + 400*S),
  -100/(S^2 + 400*S)],
 [-100/(S^2 + 400*S),
  -100/(S^2 + 400*S),
  -100/(S^2 + 400*S),
  (S + 300)/(S^2 + 400*S)]]

In [5]:
theta_m0 = [t - mu_0 for t in ts]
theta_adj = [t + logit_p1 for t in ts]
exp_theta_adj = [exp(t) for t in theta_adj]
quad_term = sum([sum([theta_m0[i] * Q[i][j] * theta_m0[j] / 2 for j in range(n_arms)]) for i in range(n_arms)]).full_simplify()

In [6]:
quad_term

1/2*(2*(S + 200)*mu_0^2 - 2*(S + 200)*mu_0*t_0 + (S + 300)*t_0^2 + (S + 300)*t_1^2 - 2*((S + 200)*mu_0 + 100*t_0)*t_1)/(S^2 + 400*S)

In [7]:
bin_term = sum([theta_adj[i] * y[i] - n[i] * log(exp_theta_adj[i] + 1) for i in range(n_arms)]).full_simplify()

In [8]:
bin_term

(L + t_0)*y_0 + (L + t_1)*y_1 - n_0*log(e^(L + t_0) + 1) - n_1*log(e^(L + t_1) + 1)

In [9]:
jll = (quad_term + bin_term)

In [10]:
M = var('M')
nt = 2
full_taylor = jll
for i in range(n_arms):
    full_taylor = full_taylor.taylor(ts[i], M, nt).full_simplify()

In [11]:
full_taylor

1/2*(2*(S*(e^(2*L + 2*M) + 2*e^(L + M) + 1) + 200*e^(2*L + 2*M) + 400*e^(L + M) + 200)*mu_0^2 - ((S^2*e^(L + M) + 400*S*e^(L + M))*n_0 - S*(e^(2*L + 2*M) + 2*e^(L + M) + 1) - 300*e^(2*L + 2*M) - 600*e^(L + M) - 300)*t_0^2 - ((S^2*e^(L + M) + 400*S*e^(L + M))*n_1 - S*(e^(2*L + 2*M) + 2*e^(L + M) + 1) - 300*e^(2*L + 2*M) - 600*e^(L + M) - 300)*t_1^2 + ((2*M*e^(2*L + 2*M) - (M^2*e^L - 2*M*e^L)*e^M - 2*(e^(2*L + 2*M) + 2*e^(L + M) + 1)*log(e^(L + M) + 1))*S^2 + 400*(2*M*e^(2*L + 2*M) - (M^2*e^L - 2*M*e^L)*e^M - 2*(e^(2*L + 2*M) + 2*e^(L + M) + 1)*log(e^(L + M) + 1))*S)*n_0 + ((2*M*e^(2*L + 2*M) - (M^2*e^L - 2*M*e^L)*e^M - 2*(e^(2*L + 2*M) + 2*e^(L + M) + 1)*log(e^(L + M) + 1))*S^2 + 400*(2*M*e^(2*L + 2*M) - (M^2*e^L - 2*M*e^L)*e^M - 2*(e^(2*L + 2*M) + 2*e^(L + M) + 1)*log(e^(L + M) + 1))*S)*n_1 - 2*((S*(e^(2*L + 2*M) + 2*e^(L + M) + 1) + 200*e^(2*L + 2*M) + 400*e^(L + M) + 200)*mu_0 - (((M*e^L - e^L)*e^M - e^(2*L + 2*M))*S^2 + 400*((M*e^L - e^L)*e^M - e^(2*L + 2*M))*S)*n_0)*t_0 - 2*((S*(e^

In [12]:
A, B = var('A', 'B')
integral = exp(full_taylor)
for i in range(1):
    integral = integrate(integral, ts[i], A, B).full_simplify()
integral

1/2*((sqrt(2)*sqrt(pi)*e^(L + M) + sqrt(2)*sqrt(pi))*erf(((A*e^(2*L + 2*M) + 2*A*e^(L + M) + A)*S - (S*(e^(2*L + 2*M) + 2*e^(L + M) + 1) + 200*e^(2*L + 2*M) + 400*e^(L + M) + 200)*mu_0 - ((((A + 1)*e^L - M*e^L)*e^M + e^(2*L + 2*M))*S^2 + 400*(((A + 1)*e^L - M*e^L)*e^M + e^(2*L + 2*M))*S)*n_0 + (S^2*(e^(2*L + 2*M) + 2*e^(L + M) + 1) + 400*S*(e^(2*L + 2*M) + 2*e^(L + M) + 1))*y_0 - 100*t_1*(e^(2*L + 2*M) + 2*e^(L + M) + 1) + 300*A*e^(2*L + 2*M) + 600*A*e^(L + M) + 300*A)/(((sqrt(2)*e^(L + M) + sqrt(2))*S^2 + 400*(sqrt(2)*e^(L + M) + sqrt(2))*S)*sqrt(((S^2*e^(L + M) + 400*S*e^(L + M))*n_0 - S*(e^(2*L + 2*M) + 2*e^(L + M) + 1) - 300*e^(2*L + 2*M) - 600*e^(L + M) - 300)/(S^2 + 400*S))))*e^(1/2*M^2*S^3*n_0^2*e^(2*L + 2*M)/(S^3*n_0*e^(3*L + 3*M) + 2*S^3*n_0*e^(2*L + 2*M) + S^3*n_0*e^(L + M) + 800*S^2*n_0*e^(3*L + 3*M) + 1600*S^2*n_0*e^(2*L + 2*M) + 800*S^2*n_0*e^(L + M) - S^2*e^(4*L + 4*M) - 4*S^2*e^(3*L + 3*M) + 160000*S*n_0*e^(3*L + 3*M) - 6*S^2*e^(2*L + 2*M) + 320000*S*n_0*e^(2*L + 2*M) - 

In [ ]:
integrate(integral, ts[1], A, B).full_simplify()